collectData.py   
Johnathan Clementi   
March, 2022   

This script gathers data used to make the Hudson River School of Art mapper and visualizer webpage

In [1]:
from bs4 import BeautifulSoup
import urllib.request
import regex as re
import requests
import pandas as pd
import numpy as np
import json
import os

In [2]:
path = os.getcwd()

# The url to pull data from:
url = "https://en.wikipedia.org/wiki/List_of_Hudson_River_School_artists"

df = pd.read_html(url)[0]
# df

In [3]:
# https://stackoverflow.com/questions/56757261/extract-href-using-pandas-read-html

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table')

linkNames = []
artistNames = []
links = []
for tr in table.findAll("tr"):
    trs = tr.findAll("td")
    for each in trs:
        try:
            linkName = each.find("a", attrs={"class":"extiw"})
            if type(linkName) != type(None):
                # linkNames.append(linkName) # Append whole BS4 object to list
                
                # Parse out links
                pageLink = re.sub("^.*href=\"","", str(linkName))
                pageLink = re.sub("\".*", "", pageLink, count=0)
                links.append(pageLink)

                pageName = re.sub("^.*Category:", "", str(linkName))
                pageName = re.sub("\">.*", "", pageName, count=0)
                pageName = re.sub(" ", "_", pageName)
                artistNames.append(pageName)

                # Create directories for each artist & the images associated with them
                outFolder = "{}\\images\\{}".format(path, pageName)
                if os.path.exists(outFolder):
                    if os.path.isdir(outFolder):
                        print('The proper output folder exists, moving on')
                    else:
                        os.mkdir(outFolder)
                        # print('Created the output directory')
                else: 
                    os.mkdir(outFolder)
                    # print('Created the output directory')

                # Download images from each Artist's page
                response = requests.get(pageLink)
                soup = BeautifulSoup(response.text, 'html.parser')
                galleryBox = soup.findAll("li", attrs={"class":"gallerybox"})

                for box in galleryBox:
                    img = box.findAll("a", attrs={"class":"image"})
                    # print(img)
                    
                    imgLink = re.sub("^.*src=\"", "", str(img), count=0)
                    imgLink = re.sub("\/thumb", "", imgLink, count=0) # Remove tag that makes img a thumbnail
                    regx_jpg = re.compile("jpg")
                    regx_png = re.compile("png")
                    if regx_jpg.search(imgLink):
                        imgLink, sp, tail = imgLink.partition("jpg") # split text at the first jpg tag
                        imgLink = imgLink + sp # Add separator back in
                    elif regx_png.search(imgLink):
                        imgLink, sp, tail = imgLink.partition("png") # split text at the first png tag
                        imgLink = imgLink + sp # Add separator back in
                    
                    imgName = re.sub("^.*commons\/.{1}\/.{2}\/", "", str(imgLink))
                    imgName = re.sub("\%2", "_", imgName)

                    imgPath = "{}\\{}".format(outFolder, imgName)

                    # Download the image
                    urllib.request.urlretrieve(imgLink, imgPath)
        except:
            pass



# linkNames = list(filter(None, linkNames)) # Remove None from list

# Compress into a df
wikiArtistPgs = pd.DataFrame(artistNames)
wikiArtistPgs.columns = ['artist_name']
wikiArtistPgs['content_link'] = links
# wikiArtistPgs['BS4_return'] = linkNames
# wikiArtistPgs

The proper output folder exists, moving on
The proper output folder exists, moving on
The proper output folder exists, moving on
The proper output folder exists, moving on
The proper output folder exists, moving on
The proper output folder exists, moving on


In [ ]:
wikiImages = []

for link in links:
  response = requests.get(link)
  soup = BeautifulSoup(response.text, 'html.parser')
  galleryBox = soup.findAll("li", attrs={"class":"gallerybox"})

  # galleryText = soup.findAll("div", attrs={"class":"gallerytext"})
  # for txt in galleryText:
    #   pageLink = txt.find("a", attrs={"class":"galleryfilename galleryfilename-truncate"})
    #   pageLink = re.sub("^.*href=\"","", str(pageLink))
    #   pageLink = re.sub("\".*", "", pageLink, count=0)
    #   pageLink = "https://commons.wikimedia.org{}".format(pageLink)
    #   print(pageLink) 


  for box in galleryBox:
    img = box.findAll("a", attrs={"class":"image"})
    # print(img)
    
    imgLink = re.sub("^.*src=\"", "", str(img), count=0)
    imgLink = re.sub("\/thumb", "", imgLink, count=0) # Remove tag that makes img a thumbnail
    regx_jpg = re.compile("jpg")
    regx_png = re.compile("png")
    if regx_jpg.search(imgLink):
      imgLink, sp, tail = imgLink.partition("jpg") # split text at the first jpg tag
      imgLink = imgLink + sp # Add separator back in
    elif regx_png.search(imgLink):
      imgLink, sp, tail = imgLink.partition("png") # split text at the first png tag
      imgLink = imgLink + sp # Add separator back in
    
    imgName = re.sub("^.*commons\/.{1}\/.{2}\/", "", str(imgLink))
    imgName = re.sub("\%2", "_", imgName)

    # Download the image
    urllib.request.urlretrieve(imgLink, imgName)




In [54]:
# How to download images from url: https://stackoverflow.com/questions/30229231/python-save-image-from-url
img_url = "https://upload.wikimedia.org/wikipedia/commons/b/bf/Baker%2C_Charles_-_Sunset_by_the_Lake_%281849%29.jpg"
urllib.request.urlretrieve(img_url, "CharlesBakerSunset.jpg")

('CharlesBakerSunset.jpg', <http.client.HTTPMessage at 0x232df65fd60>)

In [23]:
# # empty list
# data = []
# list_header = []

# header = soup.find_all("table")[0].find("tr")
  
# for items in header:
#     try:
#         list_header.append(items.get_text())
#     except:
#         continue
  
# # for getting the data 
# HTML_data = soup.find_all("table")[0].find_all("tr")[1:]
  
# for element in HTML_data:
#     sub_data = []
#     for sub_element in element:
#         try:
#             sub_data.append(sub_element.get_text())
#         except:
#             continue
#     data.append(sub_data)
  
# # Storing the data into Pandas
# # DataFrame 
# dataFrame = pd.DataFrame(data = data, columns = list_header)

In [24]:
# The url to pull data from:
url = "https://www.hudsonriverschool.org/hudsonrivervalley"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
h3s = soup.findAll('h3')
h3s = np.asarray(h3s)

C:\Users\Johnathan\AppData\Local\Temp\ipykernel_15820\1010148947.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  h3s = np.asarray(h3s)


In [25]:
sites = []

for site in h3s:
  site = str(site)
  siteName = re.search("\;\"\>.*(\<br\/\>){1}", site)
  siteName = re.sub("^.*;\">", "", str(siteName))
  siteName = re.sub("<.*", "", str(siteName))
  if siteName != "None":
    sites.append(siteName)

sitesDF = pd.DataFrame(sites)
sitesDF.columns = ['SiteName']
# sitesDF

In [26]:
subpages = soup.findAll('div', attrs={"class":"intrinsic"})
# str(subpages[2])

pgLinks = []

for page in subpages:
  page = str(page)
  pageLink = re.search("href=\"\/.*\"", page)
  pageLink = re.sub("^.*match='href=\"","", str(pageLink))
  pageLink = re.sub("\"\'>", "", pageLink)
  pageLink = "https://www.hudsonriverschool.org{}".format(pageLink)
  pgLinks.append(pageLink)

for i in range(3):
  pgLinks.pop()

# pgLinks

sitesDF['pageLink'] = pgLinks


sitesDF.to_csv('{}\\sitenames.csv'.format(path))

In [27]:
siteNames = []
siteCoords = []
long = []
lat = []

for url in pgLinks:
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')

  # Get site name from H2 elements in page
  h2s = soup.findAll('h2')
  for h2 in h2s:
    siteName = re.search("Site.*", str(h2))
    siteName = re.sub("^.*\#[0-9][0-9]*", "", str(siteName))
    siteName = re.sub("<\/h2>\'>", "", str(siteName))
    if siteName != "None":
      siteNames.append(siteName)

  # Get lat and long values from paragraph elements in the page
  ps = soup.findAll('p')
  for p in ps:
    pElement = str(p)
    coords = re.search("Site coordinates.*Long", pElement)
    latlong = re.sub("^.*match='","", str(coords))
    latlong = re.sub("\'>", "", latlong)
    if latlong != "None":
      siteCoords.append(latlong)

      lt = re.search("\: .*\, ", latlong)
      lt = re.sub("^.*match=\'\: ", "", str(lt))
      lt = re.sub(" Lat\.\, \'>", "", lt)
      lt = re.sub("\, \'>", "", lt)
      lat.append(lt)

      lng = re.search("\, .*Long", latlong)
      lng = re.sub("^.*match=\'\, ", "", str(lng))
      lng = re.sub(" Long\'>", "", lng)
      long.append(lng)

In [28]:
sitesLocDf = pd.DataFrame(siteNames)
sitesLocDf['siteCoords'] = siteCoords
sitesLocDf['lat'] = lat
sitesLocDf['long'] = long

# Write to csv
# sitesLocDf.to_csv('{}\\siteloc.csv'.format(path))

In [34]:
siteImgs = []

for url in pgLinks:
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')

  # Get site name from H2 elements in page
  # h2s = soup.findAll('h2')
  # for h2 in h2s:
  #   siteName = re.search("Site.*", str(h2))
  #   siteName = re.sub("^.*\#[0-9][0-9]*", "", str(siteName))
  #   siteName = re.sub("<\/h2>\'>", "", str(siteName))
  #   if siteName != "None":
  #     siteNames.append(siteName)

  # Get images from each subpage
  imgs = soup.findAll('img')
  imgLink = re.search("src=\".*\"{1}", str(imgs))
  siteImgs.append(imgLink)
  # print(imgLink)

In [42]:
tst = str(siteImgs[0]).split(",")
# for i in tst:
#   print(i)
#   print("#################################")

<regex.Match object; span=(68
#################################
 4821)
#################################
 match='src="//images.squarespace-cdn.com/content/v1/5cec0723a613da0001c31544/1558972846717-OQ2U9Q8SISP0PTTJGRCQ/Art+Trail+insignia+3+color.png?format=1500w"/>
#################################
 <img alt="Hudson River Art Trail" class="Header-branding-logo" src="//images.squarespace-cdn.com/content/v1/5cec0723a613da0001c31544/1558972846717-OQ2U9Q8SISP0PTTJGRCQ/Art+Trail+insignia+3+color.png?format=1500w"/>
#################################
 <img alt="Thomas_Cole_20180514_486_cover_art.jpg" src="https://images.squarespace-cdn.com/content/v1/5cec0723a613da0001c31544/1565033606712-NK99NT69U2617VHDBFNB/Thomas_Cole_20180514_486_cover_art.jpg"/>
#################################
 <img alt="Thomas_Cole_20180514_486_cover_art.jpg" class="thumb-image" data-image="https://images.squarespace-cdn.com/content/v1/5cec0723a613da0001c31544/1565033606712-NK99NT69U2617VHDBFNB/Thomas_Cole_20180514_486

In [29]:
# Images from main page
imgs = soup.findAll('img')
# imgs

[<img alt="Hudson River Art Trail" class="Mobile-bar-branding-logo" src="//images.squarespace-cdn.com/content/v1/5cec0723a613da0001c31544/1558972846717-OQ2U9Q8SISP0PTTJGRCQ/Art+Trail+insignia+3+color.png?format=1500w"/>,
 <img alt="Hudson River Art Trail" class="Header-branding-logo" src="//images.squarespace-cdn.com/content/v1/5cec0723a613da0001c31544/1558972846717-OQ2U9Q8SISP0PTTJGRCQ/Art+Trail+insignia+3+color.png?format=1500w"/>,
 <img alt="Hudson_river_skywalk.jpg" src="https://images.squarespace-cdn.com/content/v1/5cec0723a613da0001c31544/1565036033975-W4LVIANFR5N89T931N3B/Hudson_river_skywalk.jpg"/>,
 <img alt="Hudson_river_skywalk.jpg" class="thumb-image" data-image="https://images.squarespace-cdn.com/content/v1/5cec0723a613da0001c31544/1565036033975-W4LVIANFR5N89T931N3B/Hudson_river_skywalk.jpg" data-image-dimensions="2500x1875" data-image-focal-point="0.5,0.5" data-image-id="5d488e007ca5110001ba3da5" data-load="false" data-src="https://images.squarespace-cdn.com/content/v1/5c